# school bus accident

In [1]:
import pandas as pd
import numpy as np

In [2]:
# pd.set_option('display.max_rows',100)
# pd.reset_option('display.max_rows')
# pd.set_option('display.max_rows',None)
# pd.set_option('display.max_columns',None)

In [3]:
np.random.seed(42)
n = 5000

In [ ]:
# np.random.normal(mean, standard_deviation, size)
speed = np.random.normal(45,10,n)
speed

array([49.96714153, 43.61735699, 51.47688538, ..., 76.12910201,
       53.08036189, 36.51934402])

In [5]:
fatique = np.random.uniform(0,10,n)
traphic = np.random.uniform(0,1,n)
visiblity = np.random.normal (600,200,n)
break_events = np.random.randint(3,n)
school_hours = np.random.randint(5,10,n)
weather = np.random.choice(["clear","rain","fog"],size=n,p=[0.6,0.25,0.15])

In [6]:
speed = np.random.normal(0,5,n)
visiblity += np.random.normal(0,50,n) 

In [7]:
outliers = np.random.choice(n,50)
speed[outliers] = np.random.uniform(100,140,50)

In [8]:
type(outliers)

numpy.ndarray

In [9]:
df = pd.DataFrame({
    "speed_kmph":speed,
    "fatigue_score":fatique,
    "traphic_density" :traphic,
    "visiblity":visiblity,
    "break_events": break_events,
    "hour":school_hours,
    "weather":weather
})
df

,speed_kmph,fatigue_score,traphic_density,visiblity,break_events,hour,weather
0,-5.976284,1.682366,0.312656,344.240048,1913,9,rain
1,-2.300296,1.899347,0.178233,303.470933,1913,9,clear
2,3.997594,4.607122,0.859182,129.548512,1913,5,clear
3,-0.157311,2.862799,0.403523,848.204799,1913,5,rain
4,1.501434,2.474630,0.398544,781.494878,1913,8,clear
...,...,...,...,...,...,...,...
4995,0.158726,1.402850,0.636944,720.759044,1913,9,rain
4996,1.602018,8.865641,0.746460,469.199344,1913,9,fog
4997,-10.718431,7.297726,0.927614,669.926095,1913,8,clear
4998,3.449247,1.646703,0.646860,803.795794,1913,7,clear


# feature engineering

In [10]:
df["overspeeding"] = (df['speed_kmph'] > 50).astype(int)

In [13]:
df["traphic_pressure"] = df["traphic_density"]*df["speed_kmph"]
df["visiblity_risk"] = 1/(df["visiblity"] + 1)
df["rush_hours"] = df["hour"].isin([8,9,10,16,17,18]).astype(int)
df["harsh_break"] = (df["break_events"]>8).astype(int)
weather_risk = {"clear": 1,"rain" : 1.5, "fog":2} 
df["weather_risk"]= df["weather"].map(weather_risk)

In [14]:
risk_score = (
    0.3 * df["speed_kmph"] + 
    0.25 * df["fatigue_score"] +
    0.2 * df["traphic_density"] +
    0.15 * df["visiblity_risk"] + 
    0.1 * df["break_events"]
)

In [17]:
prob = 1/ (1+np.exp(-risk_score)) # 0 = no acc , 1 = acc

df["accident_risk"] = (prob>0.5).astype(int)

In [21]:
df

,speed_kmph,fatigue_score,traphic_density,visiblity,break_events,hour,weather,overspeeding,traphic_pressure,visiblity_risk,rush_hours,harsh_break,weather_risk,accident_risk
0,-5.976284,1.682366,0.312656,344.240048,1913,9,rain,0,-1.868523,0.002897,1,1,1.5,1
1,-2.300296,1.899347,0.178233,303.470933,1913,9,clear,0,-0.409989,0.003284,1,1,1.0,1
2,3.997594,4.607122,0.859182,129.548512,1913,5,clear,0,3.434662,0.007660,0,1,1.0,1
3,-0.157311,2.862799,0.403523,848.204799,1913,5,rain,0,-0.063479,0.001178,0,1,1.5,1
4,1.501434,2.474630,0.398544,781.494878,1913,8,clear,0,0.598387,0.001278,1,1,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.158726,1.402850,0.636944,720.759044,1913,9,rain,0,0.101100,0.001386,1,1,1.5,1
4996,1.602018,8.865641,0.746460,469.199344,1913,9,fog,0,1.195842,0.002127,1,1,2.0,1
4997,-10.718431,7.297726,0.927614,669.926095,1913,8,clear,0,-9.942569,0.001490,1,1,1.0,1
4998,3.449247,1.646703,0.646860,803.795794,1913,7,clear,0,2.231181,0.001243,0,1,1.0,1


In [23]:
df.to_csv("school_bus_accident_data.csv",index=False)